# Consolidating Route Patterns in GTFS

Occurs after running R script that filters the trips, schedules, etc. by a chosen date.
This part first came after an import to TransCAD but this is an alternate approach of consolidating solely based on GTFS instead of trying to join the calculated trips per Route_Name back to the route_pattern_id. There isn't a great field to join back on as we don't know exactly how TransCAD is translating the GTFS information into the Route_Names and how the routes are being consolidated as the number of unique route patterns differs between TransCAD and GTFS with GTFS having less unique route patterns in its trip table *(both post R filtering)

As discussed with Marty and Sabiheh, the goal of this consolidation step is to re-assign route patterns with low numbers of trips across the day ( less than three in each time period) to route_pattern_ids with the most trips within their matching Route, Direction, and Headsign. This gets filtered down to the trip level where if a trip's route pattern is consolidated, it will be replaced with the route_pattern_id with max trips.

All trips with same headsign/route/direction  are assigned the same route_pattern_id. The route_pattern_id that represents the group is the route_pattern_id with the most trips per day.

If less than 3 trips per tod period across all day, assign the route_pattern_id of the dir/route group with the most trips.

Calc TOD by 3-3 not 6:30 - 30:29

reorder stop times at end by trip_id

In [1]:
import time
import datetime

import matplotlib
matplotlib.use('agg')  # allows notebook to be tested in Travis

import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import pandana as pdna
import time

import urbanaccess as ua
from urbanaccess.config import settings
from urbanaccess.gtfsfeeds import feeds
from urbanaccess import gtfsfeeds
from urbanaccess.gtfs.gtfsfeeds_dataframe import gtfsfeeds_dfs
from urbanaccess.network import ua_network, load_network

%matplotlib inline

In [2]:
# required bbox including all of Massachusetts and RI as well as parts of NH, CT, NY
bbox = (-73.7207, 41.1198, -69.7876, 43.1161)
# path to the downloaded and cleaned gtfs - mbta recap file for fall 2018
#   this could also be a folder of gtfs folders (pre merge of multiple gtfs)

path_to_gtfs = r"C:\Users\matkinson.AD\Downloads\Nov15_Sandbox\Part_2_GTFS_R\mbta2018_102418_20221115"

In [3]:
loaded_feeds = ua.gtfs.load.gtfsfeed_to_df(gtfsfeed_path= path_to_gtfs,
                                           validation=True,
                                           verbose=True,
                                           bbox=bbox,
                                           remove_stops_outsidebbox=False,
                                           append_definitions=True)

Checking GTFS text file header whitespace... Reading files using encoding: utf-8 set in configuration.
GTFS text file header whitespace check completed. Took 0.11 seconds
--------------------------------
Processing GTFS feed: mbta2018_102418_20221115
The unique agency id: mbta was generated using the name of the agency in the agency.txt file.
Unique agency id operation complete. Took 0.01 seconds
Unique GTFS feed id operation complete. Took 0.00 seconds
No GTFS feed stops were found to be outside the bounding box coordinates
mbta2018_102418_20221115 GTFS feed stops: coordinates are in northwest hemisphere. Latitude = North (90); Longitude = West (-90).
Appended route type to stops
Appended route type to stop_times
--------------------------------
Added descriptive definitions to stops, routes, stop_times, and trips tables
Successfully converted ['departure_time'] to seconds past midnight and appended new columns to stop_times. Took 0.93 seconds
1 GTFS feed file(s) successfully read as 

Needs/Steps:
- Number of trips per time period per route_pattern_id
    - Midpoint time of each trip
    - Each trip classified by TOD (based on midpoint)
    - Sum of trips per TOD by route_pattern_id
- route_pattern_id with most daily trips per Route, Direction, Headsign
    - Sum all tod trips per route_pattern_id
    - grab just the max per Route, Direction, Headsign (but keep route_pattern_id)
- consolidate route patterns by Route, Direction, Headsign
    - if route_pattern_id has less than 3 trips in each of the 4 TODs, replace with max trips route_pattern_id

In [4]:
def get_start_stop_times(stop_times):    
    '''for every trip, grab the start time and stop time of the trip'''
    chocula =0 
    for trip_id in stop_times['trip_id'].unique():
        max_row = stop_times.query('trip_id==@trip_id').query('stop_sequence == stop_sequence.max()')[['trip_id','arrival_time']]
        min_row = stop_times.query('trip_id==@trip_id').query('stop_sequence == stop_sequence.min()')[['trip_id','arrival_time']]
        r2 = min_row.merge(max_row, how='left', on='trip_id', suffixes = ('_start','_end'))
        if chocula == 0:
            flintstone = pd.DataFrame(r2)
        else:
            flintstone=pd.concat([flintstone,r2])
        chocula +=1
    return(flintstone)


In [5]:
simpson = get_start_stop_times(gtfsfeeds_dfs.stop_times)

Note: the MBTA service day is 3AM - 26:59. For TOD assignment, everything that isn't between the hours of 6:30 - 19:00 is counted as NT. This addresses this issue as it includes 19:00-26:39 AND 3:00 - 6:30.

In [6]:
def assign_tod(start_stop):
    '''calculate midpoint of trip, use midpoint to assign TOD'''
    
    start_stop['at_end_dec'] = (
        (
            (start_stop['arrival_time_end'].str.split(":").str[0]).astype('int32')
            +
            ((start_stop['arrival_time_end'].str.split(":").str[1]
            ).astype('int32')/60)))
    start_stop['at_start_dec'] = (
        (
            (start_stop['arrival_time_start'].str.split(":").str[0]).astype('int32')
            +
            ((start_stop['arrival_time_start'].str.split(":").str[1]
            ).astype('int32')/60)))
    
    start_stop['midpoint'] = start_stop['at_start_dec'] + ((start_stop['at_end_dec']-start_stop['at_start_dec'])/2)
    start_stop['tod'] = np.where(start_stop['midpoint'].between(6.50,9.50),'AM', np.where(
        start_stop['midpoint'].between(9.50,15.00), 'MD', np.where(
            start_stop['midpoint'].between(15.00,19.00),'PM', 'NT' 
        )
            ) 
        )
    
    return start_stop


In [7]:
smurf = assign_tod(simpson)

In [8]:
smurf

,trip_id,arrival_time_start,arrival_time_end,at_end_dec,at_start_dec,midpoint,tod
0,37940074,19:25:00,20:03:00,20.050000,19.416667,19.733333,NT
0,37940075,06:35:00,07:03:00,7.050000,6.583333,6.816667,AM
0,37940076,07:14:00,07:53:00,7.883333,7.233333,7.558333,AM
0,37940077,11:12:00,11:50:00,11.833333,11.200000,11.516667,MD
0,37940079,05:48:00,06:25:00,6.416667,5.800000,6.108333,NT
...,...,...,...,...,...,...,...
0,CR-Weekday-Fall-18-924,20:35:00,21:08:00,21.133333,20.583333,20.858333,NT
0,CR-Weekday-Fall-18-925,21:40:00,22:19:00,22.316667,21.666667,21.991667,NT
0,CR-Weekday-Fall-18-926,22:35:00,23:10:00,23.166667,22.583333,22.875000,NT
0,CR-Weekday-Fall-18-927,22:40:00,23:20:00,23.333333,22.666667,23.000000,NT


#### Update the route_pattern_ids
- Getting duplicate ids where same number of trips for max, so choosing one arbitrarily.
- The section can be expanded to try to determine which has the most trips during peak period, but for now this works.

In [9]:
# FOR R/H/D, select RPID with most trips

    # add TOD to trip table
tod_trips = gtfsfeeds_dfs.trips.merge(smurf[['trip_id','tod']], how='left', on='trip_id')

    # get the number of trips per R/H/D & rpid
day_rpid = tod_trips.groupby(by=['route_id','trip_headsign','direction_id','route_pattern_id']).agg({'trip_id':'nunique'})
day_rpid = day_rpid.rename(columns = {'trip_id':'daily_trips'}).reset_index()

    # for each R/H/D, select just the route_pattern_id with the most daily trips
max_rpid = day_rpid.groupby(by=['route_id','trip_headsign','direction_id']).apply(lambda g: g[g['daily_trips'] == g['daily_trips'].max()])[['route_pattern_id','daily_trips']].reset_index()
max_rpid = max_rpid[['route_id','trip_headsign','direction_id','route_pattern_id']].rename(columns = {'route_pattern_id':'route_pattern_id_new'})

# because there are several cases where the max is held by two different route_pattern_ids, chose one arbitrarily
max_rpid = max_rpid.drop_duplicates(subset=['route_id','trip_headsign','direction_id'])
max_rpid

,route_id,trip_headsign,direction_id,route_pattern_id_new
0,1,Dudley,1,1-_-1
1,1,Harvard,0,1-_-0
2,10,Andrew via South Bay Center,0,10-6-0
3,10,City Point,0,10-_-0
4,10,City Point via South Bay Center,0,10-9-0
...,...,...,...,...
712,Orange,Forest Hills,0,Orange-3-0
713,Orange,Oak Grove,1,Orange-3-1
714,Red,Alewife,1,Red-3-1
715,Red,Ashmont,0,Red-1-0


In [10]:
# join trip table with the RPID with most trips (join is on R/H/D, RPID is an attribute of the table)

trips_update1 = gtfsfeeds_dfs.trips.merge(max_rpid, how='left', on=['route_id','trip_headsign','direction_id'])
# update the route_pattern_ids based on R/H/D max
    # this may be the same as original RPID as this should cover all R/H/D combos (therefore all trips)
trips_update1['route_pattern_id'] = trips_update1['route_pattern_id_new']
trips_update1 = trips_update1[gtfsfeeds_dfs.trips.columns]

# check to see if there are any duplicated trip_ids (meaning they would have multiple RPIDs, possibly from multiple max trip RPIDs)
trips_update1[trips_update1.duplicated(subset=['trip_id'], keep =False)]

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,trip_route_type,route_pattern_id,bikes_allowed,unique_agency_id,unique_feed_id,bikes_allowed_desc,wheelchair_accessible_desc


#### Now - update route/headsign/direction combos where each TOD period has less than three trips 
- take the route_pattern_id with max number of daily trips per route/direction

In [11]:
trips_update1.groupby(by=['route_pattern_id','route_id','direction_id','trip_headsign']).agg({'trip_id':'count','service_id':'nunique'})

,,,,trip_id,service_id
route_pattern_id,route_id,direction_id,trip_headsign,,
1-_-0,1,0,Harvard,114,1
1-_-1,1,1,Dudley,109,1
10-2-1,10,1,Townsend & Humboldt,1,1
10-6-0,10,0,Andrew via South Bay Center,1,1
10-9-0,10,0,City Point via South Bay Center,31,1
...,...,...,...,...,...
Orange-3-0,Orange,0,Forest Hills,153,1
Orange-3-1,Orange,1,Oak Grove,153,1
Red-1-0,Red,0,Ashmont,109,1


In [12]:
# add tod back to trips table
tod_trips2 = trips_update1.merge(smurf[['trip_id','tod']], how='left', on='trip_id')

# for each TOD and R/H/D
tod_rpid2 = tod_trips2.groupby(by=['route_id','trip_headsign','direction_id','tod']).agg({'tod':'count','route_pattern_id':'nunique','service_id':'nunique'})
tod_rpid2 = tod_rpid2.rename(columns = {'tod':'trips_per_tod'}).reset_index()

In [92]:
# select just the trips that need to be updated
tod_rpid3 = tod_rpid2[['route_id','trip_headsign','direction_id','tod','trips_per_tod']]
    # for R/H/D, get trips_per_tod separated into columns by TOD
woolf = tod_rpid3.pivot_table(index = ['route_id','trip_headsign','direction_id'], columns = ['tod'])

    # get just the trips where all 4 periods have less than 3 trips
needs_update = woolf['trips_per_tod'].reset_index().fillna(0).query('AM < 3 & MD < 3 & PM < 3 & NT < 3')

needs_update

tod,route_id,trip_headsign,direction_id,AM,MD,NT,PM
2,10,Andrew via South Bay Center,0,1.0,0.0,0.0,0.0
7,10,Townsend & Humboldt,1,1.0,0.0,0.0,0.0
9,100,Fellsway Garage via Fellsway,1,0.0,0.0,0.0,1.0
12,101,Medford Square,0,1.0,0.0,1.0,0.0
20,106,Franklin Square via Lebanon Loop,0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
656,CR-Franklin,Walpole,0,1.0,0.0,0.0,1.0
663,CR-Kingston,Plymouth,0,0.0,2.0,0.0,1.0
671,CR-Newburyport,Beverly,0,1.0,0.0,0.0,1.0
679,CR-Worcester,Ashland,0,1.0,0.0,0.0,0.0


In [97]:
# get the number of trips per R/D & rpid
day_rpid2 = tod_trips2.groupby(by=['route_id','direction_id','route_pattern_id']).agg({'trip_id':'nunique'})
day_rpid2 = day_rpid2.rename(columns = {'trip_id':'daily_trips'}).reset_index()

    # for each R/D, select just the route_pattern_id with the most daily trips
max_rpid2 = day_rpid2.groupby(by=['route_id','direction_id']).apply(lambda g: g[g['daily_trips'] == g['daily_trips'].max()])[['route_pattern_id','daily_trips']].reset_index()
max_rpid2 = max_rpid2[['route_id','direction_id','route_pattern_id']].rename(columns = {'route_pattern_id':'route_pattern_id_new'})

# because there are several cases where the max is held by two different route_pattern_ids, chose one arbitrarily
max_rpid2 = max_rpid2.drop_duplicates(subset=['route_id','direction_id'])

max_rpid2

,route_id,direction_id,route_pattern_id_new
0,1,0,1-_-0
1,1,1,1-_-1
2,10,0,10-9-0
3,10,1,10-9-1
4,100,0,100-3-0
...,...,...,...
402,Mattapan,1,Mattapan-_-1
403,Orange,0,Orange-3-0
404,Orange,1,Orange-3-1
405,Red,0,Red-3-0


In [114]:
# update the route_pattern_ids for just R/H/D combos that have < 3 trips in each of the 4 TOD periods
update_table = needs_update.merge(max_rpid2, how='left', on=['route_id','direction_id'])

update_table = update_table.drop_duplicates(subset=['route_id','direction_id','trip_headsign'])

update_table

,route_id,trip_headsign,direction_id,AM,MD,NT,PM,route_pattern_id_new
0,10,Andrew via South Bay Center,0,1.0,0.0,0.0,0.0,10-9-0
1,10,Townsend & Humboldt,1,1.0,0.0,0.0,0.0,10-9-1
2,100,Fellsway Garage via Fellsway,1,0.0,0.0,0.0,1.0,100-3-1
3,101,Medford Square,0,1.0,0.0,1.0,0.0,101-3-0
4,106,Franklin Square via Lebanon Loop,0,1.0,0.0,0.0,0.0,106-_-0
...,...,...,...,...,...,...,...,...
170,CR-Franklin,Walpole,0,1.0,0.0,0.0,1.0,CR-Franklin-0-0
171,CR-Kingston,Plymouth,0,0.0,2.0,0.0,1.0,CR-Kingston-0-0
172,CR-Newburyport,Beverly,0,1.0,0.0,0.0,1.0,CR-Newburyport-0-0
173,CR-Worcester,Ashland,0,1.0,0.0,0.0,0.0,CR-Worcester-0-0


#### Update those route_pattern_ids!

In [116]:
trips_update3 = trips_update1.merge(
    update_table[['route_id','direction_id','trip_headsign','route_pattern_id_new']], 
    how='left',on=['route_id','direction_id','trip_headsign'])

trips_update3['route_pattern_id'] = np.where(
    (
        (trips_update3['route_pattern_id_new'].isna())), 
    trips_update3['route_pattern_id'], 
    trips_update3['route_pattern_id_new'])

In [160]:
# update headsign for all trips with updated route_pattern_id
rpid_th = trips_update3.groupby(by=['route_pattern_id','trip_headsign']).agg({'trip_id':'count'}).reset_index()
rpid_th_max = rpid_th.groupby(by=['route_pattern_id']).apply(lambda g: g[g['trip_id'] == g['trip_id'].max()]).reset_index(drop=True)
rpid_th_max = rpid_th_max.drop_duplicates(subset='route_pattern_id')
trips_update4 = trips_update3.merge(
    rpid_th_max[['route_pattern_id','trip_headsign']], 
    how='left',on=['route_pattern_id'], suffixes=(None, '_to_rplce'))
trips_update4['trip_headsign'] = trips_update4['trip_headsign_to_rplce']
trips_update4

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,trip_route_type,route_pattern_id,bikes_allowed,unique_agency_id,unique_feed_id,bikes_allowed_desc,wheelchair_accessible_desc,route_pattern_id_new,trip_headsign_to_rplce
0,1,BUS42018-hbc48wk1-Weekday-02,38230147,Harvard,NaN,0,C01-21,010038,1,NaN,1-_-0,1,mbta,mbta2018_102418_20221115_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,NaN,Harvard
1,1,BUS42018-hbc48wk1-Weekday-02,38230148,Harvard,NaN,0,C01-21,010038,1,NaN,1-_-0,1,mbta,mbta2018_102418_20221115_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,NaN,Harvard
2,1,BUS42018-hbc48wk1-Weekday-02,38230154,Harvard,NaN,0,C01-16,010038,1,NaN,1-_-0,1,mbta,mbta2018_102418_20221115_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,NaN,Harvard
3,1,BUS42018-hbc48wk1-Weekday-02,38230155,Harvard,NaN,0,C01-6,010038,1,NaN,1-_-0,1,mbta,mbta2018_102418_20221115_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,NaN,Harvard
4,1,BUS42018-hbc48wk1-Weekday-02,38230157,Harvard,NaN,0,C01-20,010038,1,NaN,1-_-0,1,mbta,mbta2018_102418_20221115_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,NaN,Harvard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17650,Red,RTL42018-hms48011-Weekday-01,38066722,Alewife,NaN,1,S933_-52,933_0010,1,NaN,Red-3-1,0,mbta,mbta2018_102418_20221115_1,None,can accommodate at least one rider in a wheelc...,NaN,Alewife
17651,Red,RTL42018-hms48011-Weekday-01,38066723,Alewife,NaN,1,S933_-52,933_0010,1,NaN,Red-3-1,0,mbta,mbta2018_102418_20221115_1,None,can accommodate at least one rider in a wheelc...,NaN,Alewife
17652,Red,RTL42018-hms48011-Weekday-01,38066730,Alewife,NaN,1,S933_-43,933_0010,1,NaN,Red-3-1,0,mbta,mbta2018_102418_20221115_1,None,can accommodate at least one rider in a wheelc...,NaN,Alewife
17653,Red,RTL42018-hms48011-Weekday-01,38066735,Alewife,NaN,1,S933_-40,933_0010,1,NaN,Red-3-1,0,mbta,mbta2018_102418_20221115_1,None,can accommodate at least one rider in a wheelc...,NaN,Alewife


#### CHECKS

In [152]:
trips_update3[trips_update3['trip_id'].duplicated(keep=False)]

len(trips_update4['trip_id'].unique())
len(trips_update4['route_pattern_id'].unique())
len(gtfsfeeds_dfs.trips['route_pattern_id'].unique())


,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,trip_route_type,route_pattern_id,bikes_allowed,unique_agency_id,unique_feed_id,bikes_allowed_desc,wheelchair_accessible_desc,route_pattern_id_new


In [165]:
trips_update5 = trips_update4[gtfsfeeds_dfs.trips.columns].merge(gtfsfeeds_dfs.trips[['trip_id','route_pattern_id','trip_headsign']], how='left', on='trip_id', suffixes=(None,'_old'))

In [172]:
len(trips_update5.query('route_pattern_id != route_pattern_id_old').query('trip_headsign == trip_headsign_old'))


1624

#### Replace Trips Table

In [170]:
trips_keep_safe = gtfsfeeds_dfs.trips

In [174]:
gtfsfeeds_dfs.trips = trips_update5

### Create Generic Stop Times & Stop Sequence per Route Pattern

- Can't just change route_pattern_id as TransCAD does not use this field to combine trips into routes. There is no effect on the import.
- Working theory is that to consolidate routes one must use the stop_times.txt table as it defines the stop sequence for every trip. Theoretically, this is being used to consolidate trips into routes based on whether they have the same stop sequence.

Plan:
- Explore if stop times differ depending on TOD or if only realtime GTFS takes into account traffic.
    - For every route_pattern_id, get the average trip length (in minutes).
- For every route_pattern_id, get the average number of minutes between each pair of stops in the stop sequence.
- Replace the stop times and sequence for trips that had their route_pattern_id updated with the generic stop sequence and times per route_pattern_id created in the previous step. 
    - Keep the start time and work off of that.
    - Arrival time will equal departure time given that the difference is usually less than a minute. Will assume difference in time can be included in the minutes to next arrival time for aggregate modeling purposes.

In [175]:
trip_stop_replace = {}

gtfsfeeds_dfs.stop_times['first_stop'] =  0
gtfsfeeds_dfs.stop_times.loc[gtfsfeeds_dfs.stop_times.groupby('trip_id').stop_sequence.idxmin(),'first_stop'] = 1

for idx, row in gtfsfeeds_dfs.trips.query('(~route_pattern_id_old.isna()) & (route_pattern_id_old != route_pattern_id)').iterrows():
    tid = row['trip_id']
    rpid = gtfsfeeds_dfs.trips.query('trip_id == @tid').route_pattern_id

    all_trips = gtfsfeeds_dfs.trips.query('route_pattern_id == @rpid.iloc[0]').trip_id

    start_time = gtfsfeeds_dfs.stop_times.query('(trip_id == @tid) & (first_stop == 1)')
    all_start_times = gtfsfeeds_dfs.stop_times.query('(trip_id in @all_trips) & (first_stop == 1)')

    test_list = [[x,(abs(start_time['departure_time_sec']- all_start_times.query('trip_id == @x'))['departure_time_sec'].iloc[0])] for x in all_start_times['trip_id']]
    close = {}
    close = {sub[0]:sub[1] for sub in test_list}
        
    min_t = min(close, key=close.get)
    if tid != min_t:
        trip_stop_replace[tid] = min_t

In [ ]:
gtfsfeeds_dfs.stop_times = gtfsfeeds_dfs.stop_times.sort_values('stop_sequence')

for tid in trip_stop_replace.values():
    
    gtfsfeeds_dfs.stop_times.loc[gtfsfeeds_dfs.stop_times.loc[:,'trip_id'] == str(tid), 'time_between_stops'] = gtfsfeeds_dfs.stop_times.loc[gtfsfeeds_dfs.stop_times.loc[:,'trip_id'] == str(tid), 'departure_time_sec'].diff()

gtfsfeeds_dfs.stop_times

In [ ]:
gtfsfeeds_dfs.stop_times.query('time_between_stops > 0')

In [ ]:
stop_times = gtfsfeeds_dfs.stop_times
for trip in trip_stop_replace.keys():
    start_time = stop_times.query('(trip_id == @trip) & (first_stop == 1)')['departure_time_sec']
    # drop old stop times
    stop_times = stop_times.drop(
        stop_times.loc[stop_times['trip_id']==trip].index)
    # grab new stop times
    new_trip = trip_stop_replace[trip]
    nst = stop_times.query('trip_id == @new_trip')
    nst['trip_id'] = trip

    # replace the start time, then calculate the stop times by the departure_time_sec difference
    nst.loc[nst.loc[:,'first_stop']==1,'departure_time_sec'] = int(start_time.iloc[0])
    nst.loc[nst.loc[:,'first_stop']==1,'time_between_stops'] = int(start_time.iloc[0])
    nst['departure_time_sec'] = nst['time_between_stops'].cumsum()

    # recalc arrival/dep times
    nst['arrival_time'] = pd.to_datetime(nst['departure_time_sec'],unit='s').astype('str').str[11:19]
    nst['departure_time'] = nst['arrival_time']

    #keep only relevant columns
    nst = nst[stop_times.columns]

    stop_times = pd.concat([stop_times,nst])

In [ ]:
gtfsfeeds_dfs.stop_times = stop_times[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
       'stop_headsign', 'pickup_type', 'drop_off_type', 'timepoint',
       'checkpoint_id', 'unique_agency_id', 'unique_feed_id', 'route_type',
       'pickup_type_desc', 'drop_off_type_desc', 'timepoint_desc',
       'departure_time_sec']]


In [ ]:
gtfsfeeds_dfs.trips = gtfsfeeds_dfs.trips.drop(columns=['route_pattern_id_old'])

In [ ]:
gtfsfeeds_dfs.trips.to_csv(r"C:\Users\matkinson.AD\Downloads\Nov15_Sandbox\Part_3_Py_Consolidation\mbta2018_postpy\trips.txt",index=False)
gtfsfeeds_dfs.stop_times.to_csv(r"C:\Users\matkinson.AD\Downloads\Nov15_Sandbox\Part_3_Py_Consolidation\mbta2018_postpy\stop_times.txt",index=False)